In [ ]:
# Pracownia z analizy numerycznej, zadanie P1.16

## Metoda Newtona

In [1]:
IJulia.load("newton.jl")

In [7]:
module Newton

using Gauss
include("utils.jl")

export newton

function newton(F, J, X, solve = (A, B) -> \(A, B); ϵs = 1e-15, maxiter = 20, log = false)
  ϵ = 1.0
  i = 0
  if log
    @printf("Iteration\t|F(X)|\t\tϵ\n")
  end
  while ϵ > ϵs && i < maxiter
    JX = J(X)
    FX = F(X)
    δX = solve(JX, FX)
    ϵ  = norm(δX) / norm(X)
    if log
      @printf("%d\t\t%e\t%e\n", i, norm(FX), ϵ)
    end
    X  = X - δX
    i += 1
  end
  if log
    @printf("Final approximation:\n")
    print_array(X)
  end
end

end # Newton


Newton

In [2]:
IJulia.load("gauss.jl")

In [6]:
module Gauss

export gauss_naive!, gauss!

"""
Oblicza rozwiązanie układu równań AX = B metodą eliminacji Gaussa.

Korzysta z naiwnego algorytmu zaadaptowanego z książki Numerical Mathematics
and Computing Warda Cheney'a i Davida Kincaida
"""
function gauss_naive!(A, B)
  n = length(B)
  X = Array{eltype(A)}(n)
  for k in 1 : n-1
    for i = k + 1 : n
      mult = A[i, k] / A[k, k]
      A[i, k] = 0 
      for j in k + 1 : n
        A[i, j] -= mult * A[k, j]
      end
      B[i] -= mult * B[k]
    end
  end
  X[n] = B[n] / A[n, n]
  for i in n-1 : -1 : 1
    sum = B[i]
    for j in i + 1 : n
      sum -= A[i, j] * X[j]
    end
    X[i] = sum / A[i, i]
  end
  X
end


"""
Oblicza rozwiązanie układu równań AX = B metodą eliminacji Gaussa.

Korzysta z algorytmu skalowanego elementu głównego, przekształca
macierz wejściową A, tak że w górnej trójkątnej jej części znajduje się
jej postać schodkowa, natomiast w dolnej zapisane są użyte mnożniki
które wykorzystywane są w kroku podstawiania

Algorytm został zaadaptowany z książki Numerical Mathematics and Computing
Warda Cheney'a i Davida Kincaida
"""
function gauss!(A, B, log = false)
  n = length(B)
  l = gauss_eliminate!(A, n)
  if log
    @show A
    @show l
  end
  X = gauss_substitute!(A, B, l, n)
  if log
    @show X
  end
  X
end


"""
Implementuje krok eliminacji, zapisując mnożniki w dolnej trójkątnej
części macierzy A i zwracając permutację wierszy macierzy A
"""
function gauss_eliminate!(A, n)
  s = Array{eltype(A)}(n) # Scale vector
  l = Array{Int}(n)       # Index permutation
  for i in 1 : n
    l[i] = i
    smax = 0
    for j in 1 : n
      smax = max(smax, abs(A[i, j]))
    end
    s[i] = smax
  end
  for k in 1 : n-1
    rmax = 0
    maxidx = k
    for i in k : n
      r = abs(A[l[i], k] / s[l[i]])
      if r > rmax
        rmax = r
        maxidx = i
      end
    end
    t = l[maxidx]
    l[maxidx] = l[k]
    l[k] = t
    for i = k + 1 : n
      mult = A[l[i], k] / A[l[k], k]
      A[l[i], k] = mult 
      for j in k + 1 : n
        A[l[i], j] -= mult * A[l[k], j]
      end
    end
  end
  l
end

"""
Implementuje krok podstawiania
"""
function gauss_substitute!(A, B, l, n)
  X = Array{eltype(A)}(n) # Solution vector
  for k in 1 : n - 1
    for i in k + 1 : n
      B[l[i]] -= A[l[i], k] * B[l[k]]
    end
  end
  X[n] = B[l[n]] / A[l[n], n]
  for i in n-1 : -1 : 1
    sum = B[l[i]]
    for j in i + 1 : n
      sum -= A[l[i], j] * X[j]
    end
    X[i] = sum / A[l[i], i]
  end
  X
end

end # Gauss


Gauss

In [11]:
Gauss.gauss!([1.0 2.0; 3.0 4.0], [3.0, 4.0])

2-element Array{Float64,1}:
 -2.0
  2.5

In [10]:
include("examples.jl")

Examples

In [8]:
F, J, X = Examples.set_1()
Newton.newton(F, J, X, log = true)

Iteration	|F(X)|		ϵ
0		2.308679e+01	5.884301e+00
1		7.215002e+01	3.675183e-01
2		1.347658e+01	1.725992e-01
3		1.263553e+00	2.592261e-02
4		2.271418e-02	5.889474e-04
5		1.219716e-05	3.413509e-07
6		4.119923e-12	1.158651e-13
7		8.881784e-16	3.663005e-17
Final approximation:
[ 2.514e+00 4.322e+00 ]


In [11]:
F, J, X = Examples.set_2()
Newton.newton(F, J, X, log = true)

Iteration	|F(X)|		ϵ
0		1.562500e+00	1.076430e+00
1		1.456226e+00	2.001541e-01
2		1.548954e-01	3.428260e-02
3		3.007281e-03	7.209369e-04
4		1.250887e-06	3.004528e-07
5		2.167760e-13	5.211241e-14
6		1.110223e-16	5.551115e-17
Final approximation:
[ 1.000e+00 -7.333e-17 -1.000e+00 ]


In [13]:
F, J, X = Examples.set_3()
Newton.newton(F, J, X, log = true)

Iteration	|F(X)|		ϵ
0		1.732051e+00	5.927076e-01
1		7.281670e-01	7.379226e-02
2		3.597223e-02	4.400197e-03
3		1.253933e-04	1.534432e-05
4		1.784318e-09	2.150888e-10
5		6.661338e-16	7.605550e-17
Final approximation:
[ 1.681e+00 1.420e+00 1.178e+00 ]


In [17]:
F, J, X = Examples.generalized_rosenbrock(8)
@show X
Newton.newton(F, J, X, log = true)

X = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
Iteration	|F(X)|		ϵ
0		2.000000e+00	Inf
1		2.000000e+01	1.000000e+00
2		0.000000e+00	0.000000e+00
Final approximation:
[ 1.000e+00 1.000e+00 1.000e+00 1.000e+00 1.000e+00 1.000e+00 1.000e+00 1.000e+00 ]
